In [1]:
import pandas as pd


In [2]:
df = pd.read_csv('../../data/data_sources/CRE_19_County.csv')
df['COUNTY'] = df['COUNTY'].astype('str').str.zfill(3)
df['STATE'] = df['STATE'].astype('str')
df['FIPS'] = df['STATE']+df['COUNTY']

In [3]:
df = df.set_index('FIPS')

In [4]:
df = df.drop(columns = ['STATE','COUNTY','GEO_ID','NAME'])
df

,POPUNI,PRED0_E,PRED0_M,PRED0_PE,PRED0_PM,PRED12_E,PRED12_M,PRED12_PE,PRED12_PM,PRED3_E,PRED3_M,PRED3_PE,PRED3_PM
FIPS,,,,,,,,,,,,,
1001,55688,20573,6265,36.94,11.25,22750,6438,40.85,11.56,12365,5420,22.20,9.73
1003,221898,78622,21331,35.43,9.61,90552,22201,40.81,10.00,52724,19171,23.76,8.64
1005,22023,5024,2126,22.81,9.65,9171,2424,41.64,11.01,7828,2321,35.54,10.54
1007,20393,6280,2060,30.79,10.10,8986,2212,44.06,10.85,5127,2004,25.14,9.82
1009,57697,18189,4926,31.53,8.54,23950,5265,41.51,9.13,15558,4717,26.97,8.18
...,...,...,...,...,...,...,...,...,...,...,...,...,...
56037,41888,16977,4573,40.53,10.92,17781,4745,42.45,11.33,7130,3575,17.02,8.53
56039,23390,7250,2773,31.00,11.86,11567,2922,49.45,12.49,4573,2264,19.55,9.68
56041,20183,7744,2081,38.37,10.31,9346,2184,46.31,10.82,3093,1665,15.32,8.25


In [5]:
# use regex to split svi columns into distinct categories
cols = df.columns.to_list()
col_series = pd.Series(cols)
moe_str = r'.+M$'
moe_form = col_series.str.contains(moe_str)
moe_col = col_series[moe_form].to_list()
moe_col
# flag_str = r'^F+.'
# flag_form = col_series.str.contains(flag_str)
# flag_col = col_series[flag_form].to_list()
# val_str = r'^E_+.'
# val_form = col_series.str.contains(val_str)
# val_col = col_series[val_form].to_list()
# non_svi = col_series[~pct_form & ~flag_form & ~val_form].to_list()

['PRED0_M', 'PRED0_PM', 'PRED12_M', 'PRED12_PM', 'PRED3_M', 'PRED3_PM']

In [6]:
est_str = r'.+_E$'
est_form = col_series.str.contains(est_str)
est_col = col_series[est_form].to_list()
est_col

['PRED0_E', 'PRED12_E', 'PRED3_E']

In [7]:
pct_str = r'.+PE$'
pct_form = col_series.str.contains(pct_str)
pct_col = col_series[pct_form].to_list()
pct_col

['PRED0_PE', 'PRED12_PE', 'PRED3_PE']

In [8]:
cre_pct_df = df.drop(columns = moe_col)
cre_pct_df = cre_pct_df.drop(columns = est_col)
cre_pct_df.to_csv('../../data/cleaned_data/cre_fips.csv')

In [9]:
death = pd.read_csv('../../data/data_sources/death_data.csv', usecols = ['FIPS','3/31/21'])
death.dropna(inplace=True)
death['FIPS']=death['FIPS'].astype('int')
death['FIPS']=death['FIPS'].astype('str')
death = death.set_index('FIPS')

In [10]:
merge = cre_pct_df.join(death, how = 'left')
merge

,POPUNI,PRED0_PE,PRED12_PE,PRED3_PE,3/31/21
FIPS,,,,,
1001,55688,36.94,40.85,22.20,99
1003,221898,35.43,40.81,23.76,301
1005,22023,22.81,41.64,35.54,55
1007,20393,30.79,44.06,25.14,58
1009,57697,31.53,41.51,26.97,131
...,...,...,...,...,...
56037,41888,40.53,42.45,17.02,37
56039,23390,31.00,49.45,19.55,9
56041,20183,38.37,46.31,15.32,12


In [11]:
beds = pd.read_csv('../../data/data_sources/FIPS_hospital_beds.csv')
beds['COUNTYFIPS']=beds['COUNTYFIPS'].astype('str')
beds = beds.set_index('COUNTYFIPS')
beds

,BEDS
COUNTYFIPS,
1001,85
1003,332
1005,74
1007,35
1009,40
...,...
72125,167
72127,2596
72141,7


In [12]:
merge2 = merge.merge(beds, how='left', left_index =True, right_index=True)
merge2

,POPUNI,PRED0_PE,PRED12_PE,PRED3_PE,3/31/21,BEDS
FIPS,,,,,,
1001,55688,36.94,40.85,22.20,99,85.0
1003,221898,35.43,40.81,23.76,301,332.0
1005,22023,22.81,41.64,35.54,55,74.0
1007,20393,30.79,44.06,25.14,58,35.0
1009,57697,31.53,41.51,26.97,131,40.0
...,...,...,...,...,...,...
56037,41888,40.53,42.45,17.02,37,115.0
56039,23390,31.00,49.45,19.55,9,48.0
56041,20183,38.37,46.31,15.32,12,42.0


In [13]:
merge2.isnull().sum()

POPUNI         0
PRED0_PE       0
PRED12_PE      0
PRED3_PE       0
3/31/21        0
BEDS         628
dtype: int64

In [14]:
merge2 = merge2.fillna(0)
merge2

,POPUNI,PRED0_PE,PRED12_PE,PRED3_PE,3/31/21,BEDS
FIPS,,,,,,
1001,55688,36.94,40.85,22.20,99,85.0
1003,221898,35.43,40.81,23.76,301,332.0
1005,22023,22.81,41.64,35.54,55,74.0
1007,20393,30.79,44.06,25.14,58,35.0
1009,57697,31.53,41.51,26.97,131,40.0
...,...,...,...,...,...,...
56037,41888,40.53,42.45,17.02,37,115.0
56039,23390,31.00,49.45,19.55,9,48.0
56041,20183,38.37,46.31,15.32,12,42.0


In [15]:
dem = pd.read_csv('../../data/data_sources/dem_vote_pct.csv')
dem['county_fips'] = dem['county_fips'].astype('str')
dem = dem.set_index('county_fips')
dem.isnull().sum()

dem_pct    0
dtype: int64

In [16]:
merge3 = merge2.merge(dem, how='left', left_index=True, right_index=True)
merge3 = merge3.dropna(how='any')
merge3


,POPUNI,PRED0_PE,PRED12_PE,PRED3_PE,3/31/21,BEDS,dem_pct
FIPS,,,,,,,
1001,55688,36.94,40.85,22.20,99,85.0,27.018365
1003,221898,35.43,40.81,23.76,301,332.0,22.409030
1005,22023,22.81,41.64,35.54,55,74.0,45.788173
1007,20393,30.79,44.06,25.14,58,35.0,20.698280
1009,57697,31.53,41.51,26.97,131,40.0,9.569378
...,...,...,...,...,...,...,...
56037,41888,40.53,42.45,17.02,37,115.0,22.894957
56039,23390,31.00,49.45,19.55,9,48.0,66.599040
56041,20183,38.37,46.31,15.32,12,42.0,16.819960


In [17]:
merge3.to_csv('../../data/cleaned_data/cre_death_pct.csv')

In [30]:
sc = pd.read_csv('../../data/cleaned_data/social_context_FIPS.csv')
sc = sc.rename(columns = {'IBRC_Geo_ID':'FIPS'})
sc['FIPS'] = sc['FIPS'].astype('str')
sc = sc.set_index('FIPS')

In [36]:
merge4 = merge3.merge(sc, how='left', left_index=True, right_index=True)
merge4 = merge4.drop(['Income Mobility'], axis = 1)
merge4 = merge4.dropna(how = 'any')


In [37]:
merge4.isnull().sum()

POPUNI                0
PRED0_PE              0
PRED12_PE             0
PRED3_PE              0
3/31/21               0
BEDS                  0
dem_pct               0
Agreeableness         0
Belief In Science     0
Collectivism          0
Conflict Awareness    0
Conscientiousness     0
Empathy               0
Employment Rate       0
Entrepreneurship      0
Extraversion          0
Gender Equality       0
Hopefulness           0
Income Per Capita     0
Neuroticism           0
Openness              0
Religiosity           0
Risk Taking           0
Selflessness          0
Tolerance             0
Work Ethic            0
dtype: int64

In [39]:
merge4.to_csv('../../data/cleaned_data/cre_sc_death_pct.csv')

In [43]:
svi = pd.read_csv('../../data/cleaned_data/svi_percent_columns.csv')
svi['FIPS'] = svi['FIPS'].astype('str')
svi = svi.set_index('FIPS')
svi.columns.to_list()

['EP_POV',
 'EP_UNEMP',
 'EP_PCI',
 'EP_NOHSDP',
 'EP_AGE65',
 'EP_AGE17',
 'EP_DISABL',
 'EP_SNGPNT',
 'EP_MINRTY',
 'EP_LIMENG',
 'EP_MUNIT',
 'EP_MOBILE',
 'EP_CROWD',
 'EP_NOVEH',
 'EP_GROUPQ',
 'EPL_POV',
 'EPL_UNEMP',
 'EPL_PCI',
 'EPL_NOHSDP',
 'SPL_THEME1',
 'RPL_THEME1',
 'EPL_AGE65',
 'EPL_AGE17',
 'EPL_DISABL',
 'EPL_SNGPNT',
 'SPL_THEME2',
 'RPL_THEME2',
 'EPL_MINRTY',
 'EPL_LIMENG',
 'SPL_THEME3',
 'RPL_THEME3',
 'EPL_MUNIT',
 'EPL_MOBILE',
 'EPL_CROWD',
 'EPL_NOVEH',
 'EPL_GROUPQ',
 'SPL_THEME4',
 'RPL_THEME4',
 'SPL_THEMES',
 'RPL_THEMES',
 'EP_UNINSUR',
 'E_TOTPOP',
 'AREA_SQMI']

In [47]:
ep = ['EP_POV',
     'EP_UNEMP',
     'EP_PCI',
     'EP_NOHSDP',
     'EP_AGE65',
     'EP_AGE17',
     'EP_DISABL',
     'EP_SNGPNT',
     'EP_MINRTY',
     'EP_LIMENG',
     'EP_MUNIT',
     'EP_MOBILE',
     'EP_CROWD',
     'EP_NOVEH',
     'EP_GROUPQ',
     'EP_UNINSUR',
     'AREA_SQMI']

In [54]:
svi_ep = svi['AREA_SQMI']
svi_ep

FIPS
1001      594.443459
1009      644.830460
1013      776.838201
1015      605.867251
1017      596.560643
            ...     
48131    1793.476183
48505     998.411980
48507    1297.406535
48127    1328.884075
48247    1136.172598
Name: AREA_SQMI, Length: 3141, dtype: float64

In [55]:
merge5 = merge4.merge(svi_ep, how='left', left_index=True, right_index=True)
merge5 = merge5.dropna(how='any')
merge5

,POPUNI,PRED0_PE,PRED12_PE,PRED3_PE,3/31/21,BEDS,dem_pct,Agreeableness,Belief In Science,Collectivism,...,Hopefulness,Income Per Capita,Neuroticism,Openness,Religiosity,Risk Taking,Selflessness,Tolerance,Work Ethic,AREA_SQMI
FIPS,,,,,,,,,,,,,,,,,,,,,
1001,55688,36.94,40.85,22.20,99,85.0,27.018365,86.279655,70.833333,57.142857,...,91.163142,26168.0,77.925476,78.222354,91.106719,53.333333,82.142857,70.000000,60.380952,594.443459
1003,221898,35.43,40.81,23.76,301,332.0,22.409030,85.603337,63.268161,67.948815,...,82.484017,28069.0,77.232120,80.086368,71.771566,67.272980,75.586018,66.983549,70.972246,1589.793007
1005,22023,22.81,41.64,35.54,55,74.0,45.788173,87.711609,63.046939,70.099756,...,61.927181,17249.0,80.375206,78.783778,73.657368,76.066481,78.753019,65.170377,68.704105,885.001636
1007,20393,30.79,44.06,25.14,58,35.0,20.698280,84.830261,63.262028,72.034797,...,85.258871,18988.0,80.813736,77.837027,69.974652,75.136154,76.929754,69.859503,67.931677,622.461089
1009,57697,31.53,41.51,26.97,131,40.0,9.569378,85.548096,33.333333,80.000000,...,79.492703,21033.0,78.764620,78.193105,92.045455,57.603815,79.307632,64.953288,76.000000,644.830460
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56037,41888,40.53,42.45,17.02,37,115.0,22.894957,83.811791,68.303853,68.673956,...,82.403142,30945.0,79.384759,79.347081,68.147062,73.938691,76.390464,67.420658,70.956334,10426.975725
56039,23390,31.00,49.45,19.55,9,48.0,66.599040,82.886955,73.489916,63.115088,...,84.036899,46499.0,71.547359,80.522872,65.399695,79.598153,79.698193,70.877600,70.938645,3996.844622
56041,20183,38.37,46.31,15.32,12,42.0,16.819960,84.272810,67.029022,67.552392,...,84.089095,25636.0,78.771570,77.859042,67.603416,69.705859,73.332067,67.404487,69.299391,2081.719807


In [56]:
merge5.to_csv('../../data/cleaned_data/cre_svi_death_pct.csv')

In [64]:
cases = pd.read_csv('../../data/data_sources/case_data.csv',  usecols = ['FIPS','3/31/21'])
cases.dropna(inplace=True)
cases['FIPS']=cases['FIPS'].astype('int')
cases['FIPS']=cases['FIPS'].astype('str')
cases = cases.set_index('FIPS')
cases = cases.rename(columns={'3/31/21':'first_yr_cases'})

In [68]:
merge6 = merge5.merge(cases, how='left', left_index=True, right_index=True)
merge6= merge6.drop(['3/31/21'], axis=1)
merge6

,POPUNI,PRED0_PE,PRED12_PE,PRED3_PE,BEDS,dem_pct,Agreeableness,Belief In Science,Collectivism,Conflict Awareness,...,Income Per Capita,Neuroticism,Openness,Religiosity,Risk Taking,Selflessness,Tolerance,Work Ethic,AREA_SQMI,first_yr_cases
FIPS,,,,,,,,,,,,,,,,,,,,,
1001,55688,36.94,40.85,22.20,85.0,27.018365,86.279655,70.833333,57.142857,63.444323,...,26168.0,77.925476,78.222354,91.106719,53.333333,82.142857,70.000000,60.380952,594.443459,6589
1003,221898,35.43,40.81,23.76,332.0,22.409030,85.603337,63.268161,67.948815,63.751017,...,28069.0,77.232120,80.086368,71.771566,67.272980,75.586018,66.983549,70.972246,1589.793007,20505
1005,22023,22.81,41.64,35.54,74.0,45.788173,87.711609,63.046939,70.099756,51.165707,...,17249.0,80.375206,78.783778,73.657368,76.066481,78.753019,65.170377,68.704105,885.001636,2227
1007,20393,30.79,44.06,25.14,35.0,20.698280,84.830261,63.262028,72.034797,61.796095,...,18988.0,80.813736,77.837027,69.974652,75.136154,76.929754,69.859503,67.931677,622.461089,2542
1009,57697,31.53,41.51,26.97,40.0,9.569378,85.548096,33.333333,80.000000,63.136502,...,21033.0,78.764620,78.193105,92.045455,57.603815,79.307632,64.953288,76.000000,644.830460,6444
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56037,41888,40.53,42.45,17.02,115.0,22.894957,83.811791,68.303853,68.673956,62.498854,...,30945.0,79.384759,79.347081,68.147062,73.938691,76.390464,67.420658,70.956334,10426.975725,4022
56039,23390,31.00,49.45,19.55,48.0,66.599040,82.886955,73.489916,63.115088,61.166618,...,46499.0,71.547359,80.522872,65.399695,79.598153,79.698193,70.877600,70.938645,3996.844622,3609
56041,20183,38.37,46.31,15.32,42.0,16.819960,84.272810,67.029022,67.552392,60.175516,...,25636.0,78.771570,77.859042,67.603416,69.705859,73.332067,67.404487,69.299391,2081.719807,2128


In [69]:
merge6.to_csv('../../data/cleaned_data/cre_svi_case_pct.csv')